# Dependencies

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!pip install transformers ipywidgets
from transformers import BertTokenizer, BertForSequenceClassification, BertForTokenClassification
import torch.nn.functional as F
from IPython.display import display, HTML
import ipywidgets as widgets

!pip install transformers[torch]
! pip install -U accelerate
! pip install -U transformers
!pip install transformers
# !pip install transformers==4.28.0
!pip install sentencepiece
import nltk
nltk.download('averaged_perceptron_tagger')
import transformers
from transformers import TFBertTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification ,BertModel
!pip install tqdm
from tqdm import tqdm  # Import tqdm
from transformers import BertConfig
!pip install simpletransformers
from simpletransformers.ner import NERModel,NERArgs
import torch
import re

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 81.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 138.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Pipeline

In [2]:
from torch.distributed.distributed_c10d import new_subgroups_by_enumeration
def run_earnings_call_pipeline(new_transcript, model_ner_path , model_price_prediction_path, model_sentiment_path):
    # tokenize new transcripts
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    new_transcript = re.sub(r'\[[^\]]+\]', '', text)
    new_transcript = re.sub(r'[^a-zA-Z0-9\s%]', '', text)
    new_transcript = new_transcript.lower()

    #get the trained price prediction model
    model_price_prediction = BertForSequenceClassification.from_pretrained(model_price_prediction_path)

    price_inputs = tokenizer(new_transcript, max_length=512, truncation=True, padding='max_length',return_tensors='pt')
    price_outputs = model_price_prediction(**price_inputs)

    price_prediction =torch.nn.functional.softmax(price_outputs.logits, dim=-1)
    price_prediction = price_prediction.cpu().detach().numpy()
    print("Price precidiction is [decreae,increase]" , price_prediction,"\n")

    if price_prediction[0][0] >0.5 :
      price="Decreasing" #0
    else:
      price="Increasing" #1
##########################################################################################################################
    #make simple ner prediction
    model_ner = NERModel('bert',model_ner_path, use_cuda=False)
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(new_transcript)))
    prediction, model_output = model_ner.predict(tokens)
    # print("\n",prediction,"\n")

    metrics = []

    for sentence in prediction:
        for word_info in sentence:
            for word, label in word_info.items():
                if label == 'REVENUE' or label == 'MONEY'  or label == 'PROFIT':
                    metrics.append(word)

    print(metrics)

##########################################################################################################################

    #get the trained sentiment analysis model
    model_sentiment = BertForSequenceClassification.from_pretrained(model_sentiment_path)

    senti_inputs = tokenizer(new_transcript, max_length=512,add_special_tokens=True, padding='max_length',truncation = True,return_tensors='pt')
    senti_outputs = model_sentiment(**senti_inputs)

    sentiment_prediction= torch.nn.functional.softmax(senti_outputs.logits, dim=-1)
    sentiment_prediction = sentiment_prediction.cpu().detach().numpy()
    print("sentiment precidiction is [negative,positive]" , sentiment_prediction,"\n")


    if sentiment_prediction[0][0] >0.5:
      sentiment="Negative" #0
    else:
      sentiment="Positive" #1
###############################################################################################################################


    #generate final recommendation
    # recommendations = []

    if price=="Increasing" and sentiment == "Positive":
      recommendation = "Buy"
    elif price == "Decreasing" and sentiment =="Negative":
      recommendation = "Sell"
    else:
      recommendation = "Hold"

    # recommendations.append(recommendation)

    # Step 7: Investment Recommendation Output
    finalrecommendation = f"The stock price is :{price} and the snetiment is {sentiment}, so our recommendation is to: {recommendation}"
    return finalrecommendation

###############################################################################################################################


In [8]:
# text= "CCG revenue grew 2% year-over-year. We saw ongoing declines in the PC TAM, particularly in China and other emerging markets, which also led to greater than anticipated reductions in worldwide PC supply chain inventory with $30 Million profit. Declines in the PC segment were offset by a richer core mix and a 14th workweek.The Data Center business posted another good quarter, growing 9% over last year on strong cloud and comms service provider demand, partially offset by ongoing softness in the enterprise. The Internet of Things Group grew a remarkable 22% year-over-year, due largely to the performance of the video and retail verticals. In the Memory business, strong unit growth was offset by pricing declines, leading to revenue that was 6% lower year-over-year, and in our Security business, revenue grew 12% as this team continues to tighten its focus and execution. And finally the PSG group, formerly known as Altera, got off to a great start. It delivered a revenue of $359 million, and after adjusting for $100 million in acquisition-related accounting charges"

# text_test= "Q2 non-GAAP operating income reached 170 million, up 51% from the prior year. Non-GAAP operating margin of 25% was up nearly 700 basis points, compared to 18% last year. As discussed on last quarters call, we are executing against our investment plan. Our philosophy on investing is consistent. Well maintain continued disciplined investment to drive growth in our top line over time. Although we expect to ramp investment in the back half of this year, we still expect to exit the year with operating margins better than they were prior to our restructuring efforts. Q2 non-GAAP EPS was $0.72, an improvement of $0.28 from last year of $0.44. We ended Q2 with 6,748 employees, compared to 8,061 the year prior. Second quarter operating cash flow was 211 million, representing a 31% margin and an increase of 75% versus the same quarter a year ago. Free cash flow for the quarter was also up 74% year over year, coming in at 184 million, representing a 27% margin. As Im learning more about the business, I am impressed with our ability to consistently generate strong free cash flow. With regards to the balance sheet, we exited Q2 with more than 1.5 billion in cash, cash equivalents, and investments. We currently have approximately 725 million in convertible debt that will be maturing over the next few months. Our strong cash position is a reflection of the operating leverage in our business model that provides a strong level of flexibility. And as we think about utilizing this cash, we will do so thoughtfully, ensuring our capital is strategically deployed. Related to this, we redeployed excess capital during the quarter and repurchased 583,000 shares for approximately $30 million. We also announced today that we are increasing the size of our share repurchase program to 500 million, up from 200 million, as we continue to focus on being opportunistic, reducing dilution, and returning excess capital to shareholders, as appropriate. In addition to our share repurchase program, during the quarter, we used 40 million to pay taxes due on RSU settlements, reducing the dilutive impact of our equity programs. With that, let me turn to our Q3 and fiscal 24 guidance. To reiterate, while were pleased with our Q2 financial results, we also cautious in light of an uncertain macro environment and competitive dynamics. For the third quarter and fiscal year 24, we expect total revenue of 687 million to 691 million in Q3, or a 6% to 7% year-over-year increase; and 2.725 billion to 2.737 billion for fiscal '24, or an 8% to 9% year-over-year increase. Of this, we expect subscription revenue of 669 million to 673 million in Q3, or a 7% to 8% year-over-year increase; and 2.649 billion to 2.661 billion for fiscal 24, or an 8% to 9% year-over-year increase. For billings, we expect 668 million to 678 million in Q3, or a 1% to 3% growth rate year over year; and 2.804 billion to 2.824 billion for fiscal 24, or growth of 5% to 6% year over year. We expect non-GAAP gross margin to be 81% to 82% for both Q3 and fiscal 24. We expect non-GAAP operating margin to reach 22% to 23% for Q3 and 23% to 24% for fiscal 24. We expect non-GAAP fully diluted weighted average shares outstanding of 207 million to 212 million for both Q3 and fiscal 24. In closing, I want to thank our team for their warm welcome and their commitment to work with our customers in a challenging environment, and I look forward to connecting with many of you in the near future. We are focused on executing against the key initiatives that we believe will drive future long-term growth and expansion, as well as being mindful of that will set us up for profitable sustainable growth"
text_test = "Q2 revenue attrition ended the quarter at approximately 8%. In Q2, our non-GAAP operating margin rose to 31.6%, up 1000 basis points or more for the second quarter in a row. The margin improvement was driven by savings from our restructuring actions, strengthened revenue, and investment timing. I am very proud of our progress.Our disciplined approach to cost management has allowed us to exceed our 30% non-GAAP margin target three quarters early. Q2 operating cash flow was 808 million, up 142% year over year. Q2 free cash flow was 628 million, up 379% year over year. The upside in cash flow was driven by stronger collections than expected and lower cash outflows tied to the Q2 margin benefits we just discussed.Now, turning to remaining performance obligation. RPO, which represents all future revenue under contract, ended Q2 at 46.6 billion, up 12% year over year. Current remaining performance obligation, or CRPO, ended at 24.1 billion, up 12% year over year and 11% in constant currency. This was ahead of expectations notably due to the momentum in MuleSoft.As we called out last quarter CRPO growth did include a one-point headwind from professional services. And finally, we continue to deliver on our capital return commitment. In Q2, we returned 1.9 billion in the form of share repurchases, bringing the total return to more than $8 billion, representing more than 48 million shares since the program was initiated just last August. Before turning to guidance, I wanted to briefly touch on the current macro environment.As you heard from Brian, the measured macro environment continues to impact customer decision-making, and we are still seeing elongated sales cycles, additional deal approval layers, and deal compression in our subscription and support and professional services businesses. These factors are incorporated in our guidance. Lets start with fiscal year 24. On revenue, we are raising our guidance to 34.7 to 34.8, representing 11% growth year over year in both nominal and constant currency.The increase is driven by strength in our subscription and support revenue, particularly in the MuleSoft. We are accelerating our transformation to profitable growth. For fiscal year 24, we are pleased to raise non-GAAP operating margin guidance to 30%, representing a 750-basis-point improvement year over year. As Marc highlighted, we in a new AI era, a new innovation cycle that we will continue to invest into, as we have over the last decade.As a result, we expect nonlinear quarterly margins in the back half of this year, driven by investment timing, specifically in AI-focused R&D. We also remain focused on stock-based compensation and expected to improve this year to slightly above 8% as a percent of revenue. As a result of these updates, we now expect fiscal year 24 GAAP diluted EPS of $3.50 to $3.52, including estimated charges for the restructuring of $1.11. Non-GAAP diluted EPS is now expected to be $8.04 to $8.06.We are raising our fiscal year 24 operating cash flow growth guidance to approximately 22% to 23%. This continues to include a 14-point to 16-point headwind from restructuring. As a reminder, we expect to see an increase in our cash taxes in fiscal year 24 as we draw down our remaining net operating losses. We expect an even greater cash tax headwind next fiscal year as we draw down our remaining tax credit carry forward.Capex for the fiscal year is expected to be slightly below 2.5% of revenue. This results in free cash flow growth of approximately 24% to 25% for the fiscal year. Now, to guidance for Q3. On revenue, we expect 8.7 billion to 8.72 billion, growth of approximately 11% or 10% in constant currency."



model_price_prediction_path = "/content/gdrive/MyDrive/3owais_Capital/TextClassification_3_400_350wrmstps/Fine_tuned_Bert"
model_ner_path ="/content/gdrive/MyDrive/3owais_Capital/NER_Model_10epochs/results"
model_sentiment_path = "/content/gdrive/MyDrive/3owais_Capital/Senti_2_800_bertUncased/Fine_tuned_Bert"

run_earnings_call_pipeline(text_test, model_ner_path, model_price_prediction_path, model_sentiment_path)


Price precidiction is [decreae,increase] [[0.51950735 0.48049268]] 



  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/7 [00:00<?, ?it/s]

['[SEP]']
sentiment precidiction is [negative,positive] [[0.62384564 0.3761543 ]] 



'The stock price is :Decreasing and the snetiment is Negative, so our recommendation is to: Sell'